In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

시작 전 train_quality_data.csv 파일의 문자열 형태는 엑셀 수식변환을 사용하여 시작하였습니다.

## Train quality data -1 제외 평균

In [3]:
train_qual  = pd.read_csv('LG/train_quality_data.csv',thousands = ',')
train_qual

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,2.020110e+13,10000,05.15.2138,0.0,0,0.0,0,0,0.0,0,0,0,0,4,0,0
1,2.020110e+13,10000,05.15.2138,0.0,0,0.0,0,0,0.0,0,0,0,0,4,0,0
2,2.020110e+13,10000,05.15.2138,0.0,0,0.0,0,0,0.0,0,0,0,0,4,0,0
3,2.020110e+13,10000,05.15.2138,0.0,0,0.0,0,0,0.0,0,0,0,0,4,0,0
4,2.020110e+13,10000,05.15.2138,0.0,0,0.0,0,0,0.0,0,0,0,0,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
828619,2.020110e+13,24997,04.22.1778,0.0,0,0.0,0,0,1.0,0,0,0,0,17,0,0
828620,2.020110e+13,24997,04.22.1778,0.0,0,0.0,0,0,0.0,0,0,0,0,17,0,0
828621,2.020110e+13,24997,04.22.1778,0.0,0,0.0,0,0,3.0,0,0,0,0,17,0,0
828622,2.020110e+13,24997,04.22.1778,0.0,0,0.0,0,0,0.0,0,0,0,0,17,0,0


In [3]:
train_qual.isnull().sum()

time               0
user_id            0
fwver          40080
quality_0     144432
quality_1          0
quality_2      40113
quality_3          0
quality_4          0
quality_5         20
quality_6          0
quality_7          0
quality_8          0
quality_9          0
quality_10         0
quality_11         0
quality_12         0
dtype: int64

In [4]:
train_qual.quality_0 = train_qual.quality_0.fillna(0)
train_qual.quality_2 = train_qual.quality_2.fillna(0)
train_qual.quality_5 = train_qual.quality_5.fillna(0)

In [5]:
train_qual.isnull().sum()

time              0
user_id           0
fwver         40080
quality_0         0
quality_1         0
quality_2         0
quality_3         0
quality_4         0
quality_5         0
quality_6         0
quality_7         0
quality_8         0
quality_9         0
quality_10        0
quality_11        0
quality_12        0
dtype: int64

In [6]:
train_qual = train_qual.dropna()
train_qual.isnull().sum()

In [8]:
train_qual.fwver.unique()

array(['05.15.2138', '04.22.1750', '04.16.3553', '04.33.1261',
       '04.22.1778', '04.33.1185', '04.16.3571', '05.66.3571',
       '03.11.1149', '03.11.1167', '04.82.1684', '04.82.1778',
       '04.33.1149', '05.66.3237', '04.73.2237', '09.17.1431',
       '04.22.1684', '05.15.2120', '04.33.1125', '05.15.2122',
       '04.22.1666', '04.22.1656', '04.16.3439', '04.73.2571',
       '05.15.2114', '04.16.3345', '04.22.1442'], dtype=object)

In [9]:
train_qual

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,2.020110e+13,10000,05.15.2138,0.0,0,0.0,0,0,0.0,0,0,0,0,4,0,0
1,2.020110e+13,10000,05.15.2138,0.0,0,0.0,0,0,0.0,0,0,0,0,4,0,0
2,2.020110e+13,10000,05.15.2138,0.0,0,0.0,0,0,0.0,0,0,0,0,4,0,0
3,2.020110e+13,10000,05.15.2138,0.0,0,0.0,0,0,0.0,0,0,0,0,4,0,0
4,2.020110e+13,10000,05.15.2138,0.0,0,0.0,0,0,0.0,0,0,0,0,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
828619,2.020110e+13,24997,04.22.1778,0.0,0,0.0,0,0,1.0,0,0,0,0,17,0,0
828620,2.020110e+13,24997,04.22.1778,0.0,0,0.0,0,0,0.0,0,0,0,0,17,0,0
828621,2.020110e+13,24997,04.22.1778,0.0,0,0.0,0,0,3.0,0,0,0,0,17,0,0
828622,2.020110e+13,24997,04.22.1778,0.0,0,0.0,0,0,0.0,0,0,0,0,17,0,0


In [10]:
for x in range(16): 
    print(train_qual.iloc[:,x].value_counts())

2.020110e+13    788340
2.020100e+13       204
Name: time, dtype: int64
15357    4224
23065    3828
22632    3744
11635    3504
16147    3336
         ... 
18701      12
17675      12
17163      12
11014      12
18186      12
Name: user_id, Length: 8224, dtype: int64
05.15.2138    163236
04.22.1750    142032
04.33.1261    131340
04.16.3553    111996
03.11.1167    104148
04.33.1185     66024
04.22.1778     56472
04.22.1684      3420
09.17.1431      3384
04.16.3571      2232
04.82.1684      1044
04.33.1149       804
04.22.1666       636
05.66.3237       552
04.82.1778       216
03.11.1149       204
04.73.2237       192
04.33.1125       144
05.15.2114        96
05.66.3571        96
04.22.1442        84
05.15.2120        84
05.15.2122        60
04.73.2571        12
04.16.3439        12
04.22.1656        12
04.16.3345        12
Name: fwver, dtype: int64
 0.0       647142
-1.0       130828
 1.0         2097
 2.0         1252
 3.0          518
            ...  
 957.0          1
 3835.0       

In [11]:
train_qual_0 = train_qual.replace(-1,0)

In [12]:
train_quality_user_id_fwver_groupby = train_qual_0.groupby(['user_id', 'fwver'])[
    "quality_0","quality_1","quality_2",
    "quality_5","quality_6","quality_7","quality_8",
    "quality_9","quality_10","quality_11","quality_12"].median()

In [13]:
train_quality_fwver_groupby = train_qual_0.groupby(['fwver'])[
    "quality_0","quality_1","quality_2",
    "quality_5","quality_6","quality_7","quality_8",
    "quality_9","quality_10","quality_11","quality_12"].median()

In [17]:
for x in range(11): 
    print(train_quality_fwver_groupby.iloc[:,x].value_counts())

0.0    27
Name: quality_0, dtype: int64
0.0    27
Name: quality_1, dtype: int64
0.0    27
Name: quality_2, dtype: int64
0.0    27
Name: quality_5, dtype: int64
0.0    27
Name: quality_6, dtype: int64
0.0      24
97.0      1
5.0       1
718.0     1
Name: quality_7, dtype: int64
0.0    26
1.0     1
Name: quality_8, dtype: int64
0.0    27
Name: quality_9, dtype: int64
3.0     4
6.0     4
7.0     4
8.0     3
4.0     2
0.0     2
2.0     2
11.5    1
35.0    1
9.0     1
3.5     1
10.0    1
1.0     1
Name: quality_10, dtype: int64
0.0    27
Name: quality_11, dtype: int64
0.0    27
Name: quality_12, dtype: int64


IndexError: single positional indexer is out-of-bounds

In [15]:
train_quality_fwver_groupby

,quality_0,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
fwver,,,,,,,,,,,
03.11.1149,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
03.11.1167,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
04.16.3345,0.0,0.0,0.0,0.0,0.0,718.0,0.0,0.0,0.0,0.0,0.0
04.16.3439,0.0,0.0,0.0,0.0,0.0,5.0,1.0,0.0,2.0,0.0,0.0
04.16.3553,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0
04.16.3571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0
04.22.1442,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
04.22.1656,0.0,0.0,0.0,0.0,0.0,97.0,0.0,0.0,0.0,0.0,0.0
04.22.1666,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0


## train id fwver별로 묶을 인덱스 불러오기

In [62]:
train_err  = pd.read_csv('LG/err_group_train_target.csv')
train_err

,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,...,errtype_39,errtype_40,errtype_41,errtype_42,time_min,time_max,user_id,model_nm,fwver,time
0,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,...,0.0,0.0,0.0,0.0,20201101025616,20201130212033,10000,model_3,05.15.2138,NaN
1,0.0,0.0,0.0,0.0,9.0,1.0,1.0,0.0,0.0,0.0,...,0.0,74.0,28.0,0.0,20201101020415,20201112030039,10001,model_2,04.33.1185,2.020111e+13
2,0.0,0.0,0.0,0.0,44.0,0.0,0.0,0.0,0.0,0.0,...,0.0,39.0,28.0,1.0,20201112030617,20201130222646,10001,model_2,04.33.1261,2.020111e+13
3,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,20201101030251,20201130211528,10002,model_3,05.15.2138,NaN
4,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,20201101182532,20201110072130,10003,model_2,04.33.1185,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24841,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,38.0,0.0,0.0,20201101071624,20201119031243,24997,model_0,04.22.1750,2.020112e+13
24842,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,...,0.0,20.0,8.0,5.0,20201119031831,20201130231434,24997,model_0,04.22.1778,2.020112e+13
24843,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,20201101050758,20201103050737,24998,model_0,04.22.1750,2.020111e+13
24844,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,6.0,0.0,0.0,20201103051329,20201130210420,24998,model_0,04.22.1778,2.020111e+13


In [63]:
train_err_name = train_err.groupby(["user_id","fwver"]).sum()
train_err_name = pd.DataFrame(train_err_name.index)

In [64]:
train_err_name

,0
0,"(10000, 05.15.2138)"
1,"(10001, 04.33.1185)"
2,"(10001, 04.33.1261)"
3,"(10002, 05.15.2138)"
4,"(10003, 04.33.1185)"
...,...
24057,"(24997, 04.22.1750)"
24058,"(24997, 04.22.1778)"
24059,"(24998, 04.22.1750)"
24060,"(24998, 04.22.1778)"


In [65]:
a = pd.DataFrame(np.empty((1 ,2), object))
for x in range(len(train_err_name)):
    b = pd.DataFrame(train_err_name.iloc[x,0]).T
    a = a.append(b)

In [66]:
train_err_name = a.dropna()

In [67]:
train_err_name.rename(columns = {0 : 'user_id'}, inplace = True)
train_err_name.rename(columns = {1 : 'fwver'}, inplace = True)

In [68]:
train_err_name

,user_id,fwver
0,10000,05.15.2138
0,10001,04.33.1185
0,10001,04.33.1261
0,10002,05.15.2138
0,10003,04.33.1185
...,...,...
0,24997,04.22.1750
0,24997,04.22.1778
0,24998,04.22.1750
0,24998,04.22.1778


## Train quality data -1 제외 평균낸거 인덱싱

In [69]:
train_quality_user_id_fwver_merge = pd.merge(train_err_name,train_quality_user_id_fwver_groupby, on=['user_id','fwver'], how ='left')

In [70]:
train_quality_fwver_merge = pd.merge(train_err_name,train_quality_fwver_groupby, on=['fwver'], how ='left')

In [71]:
train_quality_user_id_fwver_merge.isnull().sum()

user_id           0
fwver             0
quality_0     14013
quality_1     14013
quality_2     14013
quality_5     14013
quality_6     14013
quality_7     14013
quality_8     14013
quality_9     14013
quality_10    14013
quality_11    14013
quality_12    14013
dtype: int64

In [72]:
train_quality_fwver_merge.isnull().sum()

user_id        0
fwver          0
quality_0     83
quality_1     83
quality_2     83
quality_5     83
quality_6     83
quality_7     83
quality_8     83
quality_9     83
quality_10    83
quality_11    83
quality_12    83
dtype: int64

In [73]:
train_quality_user_id_fwver_merge['quality_0']=train_quality_user_id_fwver_merge['quality_0'].fillna(train_quality_fwver_merge['quality_0'])
train_quality_user_id_fwver_merge['quality_1']=train_quality_user_id_fwver_merge['quality_1'].fillna(train_quality_fwver_merge['quality_1'])
train_quality_user_id_fwver_merge['quality_2']=train_quality_user_id_fwver_merge['quality_2'].fillna(train_quality_fwver_merge['quality_2'])
train_quality_user_id_fwver_merge['quality_5']=train_quality_user_id_fwver_merge['quality_5'].fillna(train_quality_fwver_merge['quality_5'])
train_quality_user_id_fwver_merge['quality_6']=train_quality_user_id_fwver_merge['quality_6'].fillna(train_quality_fwver_merge['quality_6'])
train_quality_user_id_fwver_merge['quality_7']=train_quality_user_id_fwver_merge['quality_7'].fillna(train_quality_fwver_merge['quality_7'])
train_quality_user_id_fwver_merge['quality_8']=train_quality_user_id_fwver_merge['quality_8'].fillna(train_quality_fwver_merge['quality_8'])
train_quality_user_id_fwver_merge['quality_9']=train_quality_user_id_fwver_merge['quality_9'].fillna(train_quality_fwver_merge['quality_9'])
train_quality_user_id_fwver_merge['quality_10']=train_quality_user_id_fwver_merge['quality_10'].fillna(train_quality_fwver_merge['quality_10'])
train_quality_user_id_fwver_merge['quality_11']=train_quality_user_id_fwver_merge['quality_11'].fillna(train_quality_fwver_merge['quality_11'])
train_quality_user_id_fwver_merge['quality_12']=train_quality_user_id_fwver_merge['quality_12'].fillna(train_quality_fwver_merge['quality_12'])
train_quality_user_id_fwver_merge = train_quality_user_id_fwver_merge.fillna(0)

In [74]:
train_quality_user_id_fwver_merge

,user_id,fwver,quality_0,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,10000,05.15.2138,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,6.000000,0.000000,0.000000
1,10001,04.33.1185,0.000000,0.008179,0.000000,80.585545,1.163804,13.965649,0.098146,0.000000,967.026536,0.001984,0.023810
2,10001,04.33.1261,0.000000,0.015448,0.000000,67.848987,1.849124,22.189493,0.185381,0.000000,814.187848,0.001789,0.021471
3,10002,05.15.2138,0.020833,0.000000,0.010417,0.322917,0.479167,5.750000,0.000000,0.125000,3.875000,0.000000,0.000000
4,10003,04.33.1185,0.000000,0.008179,0.000000,80.585545,1.163804,13.965649,0.098146,0.000000,967.026536,0.001984,0.023810
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24057,24997,04.22.1750,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000
24058,24997,04.22.1778,0.000000,0.000000,0.000000,1.416667,0.000000,0.000000,0.000000,0.000000,17.000000,0.000000,0.000000
24059,24998,04.22.1750,0.000000,0.007189,0.000000,117.065513,1.167983,14.015799,0.086262,0.000000,1404.786161,0.000120,0.001436
24060,24998,04.22.1778,0.000000,0.014698,0.000000,131.047369,0.599678,7.196133,0.176371,0.000000,1572.568423,0.001381,0.016575


In [75]:
train_quality_user_id_fwver_merge.to_csv('train_quality_merge.csv',index=False)

----
## Train quality data -1 갯수 세기

In [76]:
train_qul  = pd.read_csv('LG/train_quality_data.csv',thousands = ',')
train_qul

,Unnamed: 0,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,...,quality_3_minus,quality_4_minus,quality_5_minus,quality_6_minus,quality_7_minus,quality_8_minus,quality_9_minus,quality_10_minus,quality_11_minus,quality_12_minus
0,0,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
828619,828619,20201124033000,24997,04.22.1778,0.0,0,0.0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
828620,828620,20201124033000,24997,04.22.1778,0.0,0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
828621,828621,20201124033000,24997,04.22.1778,0.0,0,0.0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
828622,828622,20201124033000,24997,04.22.1778,0.0,0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
for i in range(13):
    original_column = 'quality_' + str(i)
    new_column = 'quality_' + str(i) + str('_minus')
    train_qul[new_column]=0
    for j in tqdm(range(747972)):
        if train_qul[original_column][j] == -1:
            train_qul[new_column][j] = 1
            train_qul[original_column][j] = 0

In [ ]:
train_qul

In [77]:
train_minus = train_qul.drop(["Unnamed: 0","quality_0","quality_1","quality_2","quality_3","quality_4",
    "quality_5","quality_6","quality_7","quality_8",
    "quality_9","quality_10","quality_11","quality_12","quality_3_minus", "quality_4_minus"],axis=1)

In [78]:
train_minus

,time,user_id,fwver,quality_0_minus,quality_1_minus,quality_2_minus,quality_5_minus,quality_6_minus,quality_7_minus,quality_8_minus,quality_9_minus,quality_10_minus,quality_11_minus,quality_12_minus
0,20201129090000,10000,05.15.2138,0,0,0,0,0,0,0,0,0,0,0
1,20201129090000,10000,05.15.2138,0,0,0,0,0,0,0,0,0,0,0
2,20201129090000,10000,05.15.2138,0,0,0,0,0,0,0,0,0,0,0
3,20201129090000,10000,05.15.2138,0,0,0,0,0,0,0,0,0,0,0
4,20201129090000,10000,05.15.2138,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
828619,20201124033000,24997,04.22.1778,0,0,0,0,0,0,0,0,0,0,0
828620,20201124033000,24997,04.22.1778,0,0,0,0,0,0,0,0,0,0,0
828621,20201124033000,24997,04.22.1778,0,0,0,0,0,0,0,0,0,0,0
828622,20201124033000,24997,04.22.1778,0,0,0,0,0,0,0,0,0,0,0


In [79]:
for x in range(14): 
    print(train_minus.iloc[:,x].value_counts())

20201110143000    948
20201110150000    876
20201110144000    864
20201110142000    828
20201110145000    828
                 ... 
20201103041000     12
20201123054000     12
20201113061000     12
20201129055000     12
20201108041000     12
Name: time, Length: 4319, dtype: int64
15357    4224
23065    3828
22632    3744
11635    3504
16147    3336
         ... 
11014      12
24847      12
12282      12
18186      12
18698      12
Name: user_id, Length: 8281, dtype: int64
05.15.2138    163236
04.22.1750    142032
04.33.1261    131340
04.16.3553    111996
03.11.1167    104148
04.33.1185     66024
04.22.1778     56472
04.22.1684      3420
09.17.1431      3384
04.16.3571      2232
04.82.1684      1044
04.33.1149       804
04.22.1666       636
05.66.3237       552
04.82.1778       216
03.11.1149       204
04.73.2237       192
04.33.1125       144
05.15.2114        96
05.66.3571        96
05.15.2120        84
04.22.1442        84
05.15.2122        60
04.16.3345        12
04.73.2571        1

In [80]:
train_quality_user_id_fwver_groupby_minus = train_minus.groupby(['user_id', 'fwver'])[
    "quality_0_minus","quality_1_minus","quality_2_minus",
    "quality_6_minus","quality_11_minus"].mean()

In [81]:
train_quality_fwver_groupby_minus = train_minus.groupby(['fwver'])[
    "quality_0_minus","quality_1_minus","quality_2_minus",
    "quality_6_minus","quality_11_minus"].mean()

In [82]:
train_quality_user_id_fwver_groupby_minus

,,quality_0_minus,quality_1_minus,quality_2_minus,quality_6_minus,quality_11_minus
user_id,fwver,,,,,
10000,05.15.2138,0.000000,0.000000,0.000000,0.000000,0.000000
10002,05.15.2138,0.020833,0.020833,0.020833,0.020833,0.020833
10004,04.22.1750,0.083333,0.083333,0.083333,0.083333,0.083333
10005,04.22.1750,0.416667,0.416667,0.416667,0.416667,0.416667
10006,04.16.3553,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...
24992,04.22.1750,0.333333,0.333333,0.333333,0.333333,0.333333
24993,05.15.2138,0.291667,0.291667,0.291667,0.291667,0.291667
24995,04.33.1261,0.416667,0.416667,0.416667,0.416667,0.416667


In [83]:
train_quality_fwver_groupby_minus

,quality_0_minus,quality_1_minus,quality_2_minus,quality_6_minus,quality_11_minus
fwver,,,,,
03.11.1149,0.000000,0.009804,0.009804,0.009804,0.009804
03.11.1167,0.000000,0.130219,0.130219,0.130219,0.130219
04.16.3345,0.000000,0.000000,0.000000,0.000000,0.000000
04.16.3439,0.000000,0.000000,0.000000,0.000000,0.000000
04.16.3553,0.172872,0.172863,0.172872,0.172863,0.172863
04.16.3571,0.165771,0.165771,0.165771,0.165771,0.165771
04.22.1442,0.166667,0.166667,0.166667,0.166667,0.166667
04.22.1656,0.000000,0.000000,0.000000,0.000000,0.000000
04.22.1666,0.003145,0.003145,0.003145,0.003145,0.003145


In [84]:
train_id_fwver_merge_minus = pd.merge(train_err_name,train_quality_user_id_fwver_groupby_minus, on=['user_id','fwver'], how ='left')

In [85]:
train_fwver_merge_minus = pd.merge(train_err_name,train_quality_fwver_groupby_minus, on=['fwver'], how ='left')

In [86]:
train_id_fwver_merge_minus

,user_id,fwver,quality_0_minus,quality_1_minus,quality_2_minus,quality_6_minus,quality_11_minus
0,10000,05.15.2138,0.000000,0.000000,0.000000,0.000000,0.000000
1,10001,04.33.1185,NaN,NaN,NaN,NaN,NaN
2,10001,04.33.1261,NaN,NaN,NaN,NaN,NaN
3,10002,05.15.2138,0.020833,0.020833,0.020833,0.020833,0.020833
4,10003,04.33.1185,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
24057,24997,04.22.1750,0.333333,0.333333,0.333333,0.333333,0.333333
24058,24997,04.22.1778,0.000000,0.000000,0.000000,0.000000,0.000000
24059,24998,04.22.1750,NaN,NaN,NaN,NaN,NaN
24060,24998,04.22.1778,NaN,NaN,NaN,NaN,NaN


In [87]:
train_id_fwver_merge_minus['quality_0_minus']=train_id_fwver_merge_minus['quality_0_minus'].fillna(train_fwver_merge_minus['quality_0_minus'])
train_id_fwver_merge_minus['quality_1_minus']=train_id_fwver_merge_minus['quality_1_minus'].fillna(train_fwver_merge_minus['quality_1_minus'])
train_id_fwver_merge_minus['quality_2_minus']=train_id_fwver_merge_minus['quality_2_minus'].fillna(train_fwver_merge_minus['quality_2_minus'])
train_id_fwver_merge_minus['quality_6_minus']=train_id_fwver_merge_minus['quality_6_minus'].fillna(train_fwver_merge_minus['quality_6_minus'])
train_id_fwver_merge_minus['quality_11_minus']=train_id_fwver_merge_minus['quality_11_minus'].fillna(train_fwver_merge_minus['quality_11_minus'])

In [88]:
train_id_fwver_merge_minus = train_id_fwver_merge_minus.fillna(0)

In [89]:
train_id_fwver_merge_minus.to_csv('train_quality_merge_minus.csv',index=False)

In [90]:
train_id_fwver_merge_minus

,user_id,fwver,quality_0_minus,quality_1_minus,quality_2_minus,quality_6_minus,quality_11_minus
0,10000,05.15.2138,0.000000,0.000000,0.000000,0.000000,0.000000
1,10001,04.33.1185,0.190007,0.190007,0.190007,0.190007,0.190007
2,10001,04.33.1261,0.216827,0.216827,0.216827,0.216827,0.216827
3,10002,05.15.2138,0.020833,0.020833,0.020833,0.020833,0.020833
4,10003,04.33.1185,0.190007,0.190007,0.190007,0.190007,0.190007
...,...,...,...,...,...,...,...
24057,24997,04.22.1750,0.333333,0.333333,0.333333,0.333333,0.333333
24058,24997,04.22.1778,0.000000,0.000000,0.000000,0.000000,0.000000
24059,24998,04.22.1750,0.193625,0.193625,0.193625,0.193625,0.193625
24060,24998,04.22.1778,0.198594,0.198594,0.198594,0.198594,0.198594
